# AbyU OpenMM

![AbyU!](AbyU.png)

Now we will perform an ML/MM reaction simulation, specifically of the reaction catalysed by AbyU.

In this instance, we have a pretrained MACE MLP (trained to the M06-2X/6-31G* level) and use the generic embedding model. To perform this reaction we use a generalised distance restraint between two bonds, so initially we define a restraint on these bonds and add it as a custom bond in OpenMM. 

In [ ]:
import math
import os
import sys

import numpy as np

import openmm
import openmm.unit as unit
from openmm.app import *
from openmm import CustomBondForce, CustomCVForce

from emle.models import MACEEMLE
from emle.calculator import EMLECalculator

import sire as sr

distances = ((2125, 2094, 0.7), (2119, 2087, 0.3))
k = (100*unit.kilocalorie_per_mole/unit.angstrom**2).value_in_unit(unit.kilojoule_per_mole/unit.nanometer**2)
r0 = 2.5*unit.angstroms

cv = CustomBondForce("weight*r") 
cv.addPerBondParameter("weight") 

for atom1, atom2, weight in distances: 
    cv.addBond(atom1, atom2, [weight]) 

energy_expression = "k*(weighted_distance - r0)^2"
restraint_force = openmm.CustomCVForce(energy_expression)
restraint_force.addCollectiveVariable("weighted_distance", cv)
restraint_force.addGlobalParameter("k", k)
restraint_force.addGlobalParameter("r0", r0)

# Creating the system

Much like the ADP system, we initially load the system in Sire and OpenMM. The main difference here is in the creation of the calculator, as we're using a MACE MLP instead of the default ANI-2x so we tell the calculator to load a different backend and give it the model file (which you can see in the lift of files on the left). (You will see warnings as the calculator loads but that is okay!)

In [ ]:
#Loads AbyU system
mols = sr.load([f"AbyU_OpenMM.rst", f"AbyU_OpenMM.prm7"])

# Load the topology with OpenMM too.
prm = AmberPrmtopFile(f"AbyU_OpenMM.prm7")

#Create calculator
calculator = EMLECalculator(device="cpu", mace_model="models/abyu_mace.model", backend="mace")

# Create an EMLEEngine bound to the calculator on the substrate.
mols, engine = sr.qm.emle(mols, mols[1], calculator)


# Running the simulation

This is again similar to what you previously saw in the ADP example, but here we have to add a few extra settings. We have a couple new settings for the dynamics object, most obviously is the fixed atoms. This settings emulates the iBelly restraints found in AMBER, fixing all atoms outside of a 20A radius of the substrate in place. This is done for simulation efficiency. 

We also add some lines in the OpenMM context, specifically to add the biasing potentials to the system. The actual production cell runs the same as in the ADP example. 

In [ ]:
# Create a QM/MM dynamics object.
d = mols.dynamics(
    timestep="1fs",
    constraint="none",
    integrator="langevin_middle",
    temperature="300K",
    qm_engine=engine,
    map={"threads": 1},
    fixed="not atoms within 20A of atomidx 3 in mol[1]"
)

In [ ]:
# Get the underlying OpenMM context.
context = d._d._omm_mols

# Get the OpenMM system.
omm_system = context.getSystem()

# Store a copy of the integrator.
integrator = context.getIntegrator().__copy__()

# Add the forces to the OpenMM system.
omm_system.addForce(restraint_force)

# Create a new context.
new_context = openmm.Context(omm_system, integrator, context.getPlatform())

# Set force constant K for the biasing potential.
new_context.setParameter("k", k)

#Set center of biasing potential
new_context.setParameter("r0", r0)

# Set the postions of the new context to be the same as the original context.
new_context.setPositions(context.getState(getPositions=True).getPositions())



In [ ]:
# Sampling production. Trajectories are saved in dcd files.
file_handle = open(f"./AbyU_OpenMM/AbyU_OpenMM.dcd", "bw")
state = new_context.getState(getPositions=True)
positions = state.getPositions()
dcd_file = DCDFile(file_handle, prm.topology, dt=integrator.getStepSize())
dcd_file.writeModel(positions)

for x in range(300):
    integrator.step(1)
    state = new_context.getState(getPositions=True)
    positions = state.getPositions()
    dcd_file.writeModel(positions)
        
file_handle.close()

# Viewing the trajectory

Here we use NGLview to again view the trajectory. Running the cell as is views just the substrate, whilst swapping which trajectory viewing commands are edited out will allow you to view the whole system being simulated.

In [ ]:
traj = sr.load("AbyU_OpenMM.prm7", "AbyU_OpenMM/AbyU_OpenMM.dcd")

#To view the whole system
#traj.trajectory().view()

# To view just the substrate
traj[1].trajectory().view()